In [ ]:
from mantid.simpleapi import *
from mantid.api import MatrixWorkspace
from mantid.api import AnalysisDataService as ADS
from mantid.api import *
from mantid.kernel import *
from sansdata import *

## 1. Create `Workspace` from data and instrument definition

In [ ]:
# sans_files = []
# for i in range(1, 4 + 1):
#     sans_file = SansData(f"data/memb_BS_Q{i}_6_0Ang.mpa")
#     # sans_file = SansData(f"data/empty_BS_Q{i}_6_0Ang.mpa")
#     sans_files.append(sans_file)
#     sans_file.plot_2d()
# del sans_files

In [ ]:
sans_file = SansData(f"data/memb_BS_Q2_6_0Ang.mpa")
background = SansData(f"data/09_07_24_backG_3600s_reactor_on_Fish_on.mpa")
direct = SansData(f"data/empty_no_BS_Q2_Boron.mpa")
transmission = SansData(f"data/memb_no_BS_Q2_Boron_8_5Ang.mpa")
relative_pixel_efficiency = np.loadtxt('pixel-efficiency.txt.gz')

sans_file.plot_2d(True)
transmission.plot_2d(True)
direct.plot_2d(True)
background.plot_2d(True)

In [ ]:
# sans_file = sans_files[3]
# sans_file = SansData("data/Cnano_BS_Q1_6_0Ang.mpa")

In [ ]:
def workspace_from_sansdata(sans_file, background_file, pixel_efficiencies):
    delta_L_over_L0 = 0.1
    bin_lower = sans_file.L0 * (1.0 - delta_L_over_L0/2)
    bin_upper = sans_file.L0 * (1.0 + delta_L_over_L0/2)
    detectors = 1024 * 1024
    # Use the same bin for each detector
    x = np.tile([bin_lower,bin_upper],detectors)
    y = sans_file.I - background_file.I

    ws = CreateWorkspace(OutputWorkspace = sans_file.filename,UnitX="Wavelength",DataX=x, DataY=y, NSpec=detectors)

    # Create a workspace to contain the pixel efficiencies
    pixel_y = relative_pixel_efficiency
    pixel_y[pixel_y<=0] = 1
    pixel_adj = CreateWorkspace(OutputWorkspace = 'PixelAdj',UnitX="Wavelength",DataX=x, DataY=pixel_y, NSpec=detectors)

    mon = LoadInstrument(ws, FileName="RIDSANS_Definition.xml", RewriteSpectraMap=True)
    MoveInstrumentComponent(ws, ComponentName="sample-position", X=0.0, Y=0.0, Z=-sans_file.d, RelativePosition = False)
    return ws, pixel_adj, mon
ws, pixel_adj, _ = workspace_from_sansdata(sans_file,background, relative_pixel_efficiency)
ws, pixel_adj



## 2. Minimal reduction

Perform a very minimal reduction process:
- Mask beamstop and everything outside of active detector region
- 

## 2.1. Mask detectors outside of active region


In [ ]:
half_w = active_w/2
half_h = active_h/2

def mask_rectangle(ws, w, h, negative=False, offset_x = 0, offset_y=0):
    # Gets large for a 1024 x 1024 detector but at most ~30 MB
    mask_list = []
    for i in range(ws.getNumberHistograms()):
        detector = ws.getDetector(i)
        # Get the position of the detector
        position = detector.getPos()
        if (abs(position.getX() - offset_x) > w/2 or abs(position.getY() - offset_y) > h/2) == (not negative): 
            mask_list.append(i+1)
    MaskDetectors(Workspace=ws, SpectraList=mask_list)

mask_rectangle(ws, active_w, active_h)
# mask_rectangle(ws, sans_file.beamstop.w, sans_file.beamstop.h, offset_x = sans_file.beamstop.x, offset_y=sans_file.beamstop.y, negative=True)
# # Calling MaskDetectors
# mask_list = []
# for i in range(ws.getNumberHistograms()):
#     detector = ws.getDetector(i)
#     # Get the position of the detector
#     position = detector.getPos()
#     if abs(position.getX()) > half_w or abs(position.getY()) > half_h: 
#         mask_list.append(i+1)

# MaskDetectors(Workspace=ws, SpectraList=mask_list)
print("Completed masking")

In [ ]:
# Idea: move detector based on beamstop position to compensate for shift
# Question: does this actually make sense?
MoveInstrumentComponent(ws, ComponentName="detector-bank", X=-sans_file.beamstop.x, Y=-sans_file.beamstop.y, Z=0.0, RelativePosition = False)

## 2.2. Calculate $Q_{max}$ and run `Qxy` algorithm

In [ ]:
# max_det_x = 0.140662/2
# max_det_y = 0.140662/2
ds_dist = sans_file.d
r = active_w/2
Q_max = 4 * np.pi / sans_file.L0 * np.sin(np.arctan(r/(ds_dist)) / 2)
Q_max # AA-1

In [ ]:
delta_Q = 0.0001
# max_QXY = 0.01
# N_Q_bins = int(np.floor(2*Q_max/delta_Q)+2)

Qxy(ws,PixelAdj = pixel_adj, SolidAngleWeighting=False,MaxQxy=Q_max, DeltaQ=delta_Q,OutputWorkspace="new")
reduced_ws = ADS.retrieve('new')
Q_axis = np.array(reduced_ws.dataX(0))
N_Q_bins = len(Q_axis) - 1
Q_array = np.zeros((N_Q_bins, N_Q_bins))
for i in range(N_Q_bins):
    Q_array[i] = reduced_ws.dataY(i)
# Q_axis, Q_array

In [ ]:
from matplotlib.colors import LogNorm
# print(Q_array)
extent = [Q_axis[0], Q_axis[-1], Q_axis[0], Q_axis[-1]]
plt.imshow(Q_array, cmap='viridis', extent=extent, norm=LogNorm(),aspect='auto')  # cmap defines the color map (optional)
plt.colorbar()  # Add a colorbar to show the color scale
plt.xlabel(r'$Q_x$ [Å$^{-1}$]')
plt.ylabel(r'$Q_y$ [Å$^{-1}$]')
plt.xticks(fontsize=8)
plt.axvline(0, linestyle='--', color='red')
plt.axhline(0, linestyle='--', color='red')
plt.yticks(fontsize=8)
plt.show()

## 3. Inspection of workspace properties

In [ ]:
# Print some basic instrument and workspace information
inst = ws.getInstrument()
si = ws.spectrumInfo()
di = ws.detectorInfo()
ci = ws.componentInfo()
print("Workspace {0} has instrument: {1}".format(ws.name(), inst.getName()))
print("Instrument {0} has {1} components, including {2} monitors and {3} detectors".format(inst.getName(), ci.size(), len(mon), di.size()))


unit = ws.getAxis(0).getUnit().unitID()
unit2 = ws.getAxis(1).getUnit().unitID()
print(unit)
print(unit2)

# Get the sample position
instrument = ws.getInstrument()

sample = instrument.getSample()
sample_position_1 = sample.getPos()

# sample_position_2 = si.samplePosition()

# print(spectrum_info, detector_info)
print("Sample Position:", sample_position_1)
# print("Sample Position:", sample_position_2)

num_spec = ws.getNumberHistograms()
num_bins = ws.blocksize()
print("Detector spectrum blocksize:", num_bins)
print("Number of spectra:", num_spec)
for i in range(num_spec):
    # Check that all spectra have a matching detector
    assert(si.hasDetectors(i))
    # Assert that at this point no spectrum is masked
    # assert(not si.isMasked(i))
    # print(si.position(i))

The following code probes the histograms of a cross section of the detectors, looking for detectors with a particularly high count. It also verifies that the bin (X-axis) is correct.

In [ ]:
print("number of histograms = {0}".format(ws.getNumberHistograms()))
# ws.getSpectrum(3)
for i in range(0,ws.getNumberHistograms(),100):
    sum_counts = ws.readY(i)[0]

    # Display spectrum number against sum_counts
    if sum_counts >= 200.0:
        print("{0} {1} {2}".format(ws.getSpectrum(i).getSpectrumNo(), sum_counts,ws.readX(i)))